In [ ]:
import numpy as np

A = 5. * (1.e-3)**2
r = 175.e-3
psi = 10e3 * (5.e-3 * 10.e-3)

O = 4 * np.pi * r**2 / 2.

psi_act = psi / O * A
print(psi_act, psi_act * 0.03)


In [363]:
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

import numpy as np
import dat_lists
import webapi_access as api
import importlib

import matplotlib as mpl
import matplotlib.pyplot as p

XPID = '20180920.032'
program_info, req = api.xpid_info(program=XPID)
start = str(program_info['programs'][0]['from'])
stop = str(program_info['programs'][0]['upto'])

t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

names = [
    'AEH30 control', 'AEH30 Kp', 'AEH30 Ki', 'AEH30 Kd',   # 0 - 3
    'AEH51 control', 'AEH51 Kp', 'AEH51 Ki', 'AEH51 Kd',  # 4 - 7
    'ECRH', 'BoloRealTime_P_rad', 'Prad HBC', 'Prad VBC',  # 8 - 11
    'QSQ Feedback AEH30', 'QSQ Feedback AEH51']  # 12 - 13

L = []
for name in names:
    foo = api.download_single(
        api.download_link(name=name),
        program_info=program_info,
        start_POSIX=start, stop_POSIX=stop)
    L.append(np.array([
        [(t - t0) / 1e9 for t in foo['dimensions']],
        foo['values']]))

AEH30Q = L[0]
AEH51Q = L[4]
ECRH = L[8]
RTFeedback = L[9]
HBC = L[10]
VBC = L[11]

Kp_AEH51 = L[5]
Ki_AEH51 = L[6]
Kd_AEH51 = L[7]

Kp_AEH30 = L[5]
Ki_AEH30 = L[6]
Kd_AEH30 = L[7]

foo = np.array([RTFeedback[0], RTFeedback[1][0]])
ind = np.where((foo[0] > .0) & (foo[0] < T0))[0]
RTF = np.array([foo[0][ind], foo[1][ind]])

In [364]:
import matplotlib.pyplot as p

t4 = 32.
if False:  # AEH30
    # p.plot(L[0][0], L[0][1], c='k')  # control
    p.plot(L[1][0], L[1][1] / 1., c='r')  # Kp
    p.plot(L[2][0], L[2][1] / 1., c='b')  # Ki
    p.plot(L[3][0], L[3][1] / 1., c='g')  # Kd
    p.plot(L[12][0], L[12][1], c='grey')  # valve

if True:  # AEH51
    # p.plot(L[4][0], L[4][1], c='k', ls='-.')  # control
    p.plot(L[5][0], L[5][1] / 400., c='k', ls='-.')  # Kp
    p.plot(L[6][0], L[6][1] / 10., c='r', ls='-.')  # Ki
    p.plot(L[7][0], L[7][1], c='b', ls='-.')  # Kd
    p.plot(L[13][0], L[13][1] / 10., c='grey')  # valve

# p.plot(HBC[0], np.array(HBC[1][0]) / 1.e3, c='purple', ls='-.')
# p.plot(RTFeedback[0], RTFeedback[1][0], c='orange', ls='-.')
# p.plot(L[10][0], np.array(L[10][1][0]) / 1.e3, c='orange')

p.xlim(-.1, t4)

(-0.1, 32.0)

In [242]:
import time

class PID:
    """ PID Controller
    """

    def __init__(self, P=0.2, I=0.0, D=0.0, current_time=None):

        self.Kp = P
        self.Ki = I
        self.Kd = D

        self.sample_time = 0.00
        self.current_time = current_time if current_time is not None else time.time()
        self.last_time = self.current_time

        self.clear()

    def clear(self):
        """Clears PID computations and coefficients"""
        self.SetPoint = 0.0

        self.PTerm = 0.0
        self.ITerm = 0.0
        self.DTerm = 0.0
        self.last_error = 0.0

        # Windup Guard
        self.int_error = 0.0
        self.windup_guard = 20.0

        self.output = 0.0

    def update(self, feedback_value, current_time=None,
               Kp_in=None, Ki_in=None, Kd_in=None):
        """ Calculates PID value for given reference feedback
                u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}
            Test PID with Kp=1.2, Ki=1, Kd=0.001 (test_pid.py)
        """
        error = self.SetPoint - feedback_value

        self.current_time = current_time if current_time is not None else time.time()
        delta_time = self.current_time - self.last_time
        delta_error = error - self.last_error

        if (delta_time >= self.sample_time):
            # prop
            self.PTerm = Kp_in if Kp_in is not None else error
            
            # ineg
            self.ITerm += error * delta_time
            if (self.ITerm < -self.windup_guard):
                self.ITerm = -self.windup_guard
            elif (self.ITerm > self.windup_guard):
                self.ITerm = self.windup_guard
            self.ITerm = Ki_in if Ki_in is not None else self.ITerm

            # diff
            self.DTerm = 0.0
            if delta_time > 0:
                self.DTerm = Kd_in if Kd_in is not None else delta_error / delta_time

            # Remember last time and last error for next calculation
            self.last_time = self.current_time
            self.last_error = error

            self.output = self.Kp * self.PTerm + (self.Ki * self.ITerm) + (self.Kd * self.DTerm)

    def setKp(self, proportional_gain):
        """Determines how aggressively the PID reacts to the current error with setting Proportional Gain"""
        self.Kp = proportional_gain

    def setKi(self, integral_gain):
        """Determines how aggressively the PID reacts to the current error with setting Integral Gain"""
        self.Ki = integral_gain

    def setKd(self, derivative_gain):
        """Determines how aggressively the PID reacts to the current error with setting Derivative Gain"""
        self.Kd = derivative_gain

    def setWindup(self, windup):
        """Integral windup, also known as integrator windup or reset windup,
        refers to the situation in a PID feedback controller where
        a large change in setpoint occurs (say a positive change)
        and the integral terms accumulates a significant error
        during the rise (windup), thus overshooting and continuing
        to increase as this accumulated error is unwound
        (offset by errors in the other direction).
        The specific problem is the excess overshooting.
        """
        self.windup_guard = windup

    def setSampleTime(self, sample_time):
        """PID that should be updated at a regular interval.
        Based on a pre-determined sampe time, the PID decides if it should compute or return immediately.
        """
        self.sample_time = sample_time

In [50]:
import main as prad_main
import prad_calculation as prad_calc

data = prad_main.main(
    program=XPID,
    filter_method='mean',
    geom_input='self',
    strgrid='sN8_30x20x150_1.3',
    magconf='EIM_beta000',
    Archive=False)

prio = data['prio']
power = data['radpow']['power']
timep = data['radpow']['time']
info = data['prio']['geometry']

chans = [65, 73, 78, 86, 61]  # chans = [14, 16, 7, 3, 0]
rad, vol = prad_calc.calculate_prad(
    time=timep, power=power,
    volume=prio['geometry']['geometry']['vbolo'],
    k_bolo=prio['geometry']['geometry']['kbolo'],
    volume_torus=prio['volume_torus'],  channels=chans,
    camera_list=chans, date=20181010, shotno=32,
    brk_chan=info['channels']['droplist'], camera_tag='none',
    saving=False, method='mean', debug=True)

rtprad = np.array([timep, (rad / 1.e6) / 1.3])


>> Start routine for P_rad calculation and diagnostics ...
	>> Selecting date ...
	>> Comparison info for shots ...
	>> Get info from logbook ...
	>> Current date is  20181010 ...
	>> SUCCESS: Server returns shot info of 20181010.032
	   NAME: #142598 ID07_O2_55MJ_9.1s_PrefillOnly
	   DESC.:  
	>> Get bolometer data and everything else...
		\\ timing error, mode: lab
		\\ 1.562%, new sample time:1.625ms
		\\ shift around T1/T4: 131.25 / 281.275 ms
		\\  <class 'TypeError'> webapi_access.py 109 stat  True  -- i --  1
	>> Finished check_bolo_data at 1
	>> Dump Run-Once stuff to priority_data...
		\\ using own geometry input: EIM_beta000 sN8_30x20x150_1.3
\\share.ipp-hgw.mpg.de\documents\pih\Documents\git\QSB_Bolometry\libprad
		>> Mean fitted calibration values are:
 			kappa_m= 6.128e-04 A^2; kappa_r= 6.167e-04 A^2
 			tau_m= 112.80 ms; tau_r= 110.47 ms;
 			roh_m= 996.86 Ohm; roh_r= 1007.60 Ohm;
	>> power and voltage: 0... 1... 2... 3... 4... 5... 7... 8... 9... 10... 11... 
		 12... 

In [213]:
rtprad = np.array([timep, (data['radpow']['P_rad_hbc'] + data['radpow']['P_rad_vbc']) / 2.e6])

In [235]:
T0 = 8.5

ind = np.where((rtprad[0] > .0) & (rtprad[0] < T0))[0]
RTF = np.array([rtprad[0][ind], rtprad[1][ind]])

ind = np.where((ECRH[0] > .0) & (ECRH[0] < T0))[0]
ECH = ECRH[:, ind]

ind = np.where((AEH30Q[0] > .0) & (AEH30Q[0] < T0))[0]
Q = AEH30Q[:, ind]

ind = np.where((Kp_AEH51[0] > .0) & (Kp_AEH51[0] < T0))[0]
Kp = Kp_AEH51[:, ind]
Ki = Ki_AEH51[:, ind]
Kd = Kd_AEH51[:, ind]

eN = int((np.shape(ECH)[1] / np.shape(Kp)[1]))
rN = int((np.shape(RTF)[1] / np.shape(Kp)[1]))

NECH, NRTF = np.zeros((2, np.shape(Kp)[1])), np.zeros((2, np.shape(Kp)[1])),
for i in range(1, np.shape(Kp)[1]):
    NECH[0, i] = np.mean(ECH[0, (i - 1) * eN:(i + 1) * eN - 1])
    NECH[1, i] = np.mean(ECH[1, (i - 1) * eN:(i + 1) * eN - 1])

    NRTF[0, i] = np.mean(RTF[0, (i - 1) * rN:(i + 1) * rN - 1])
    NRTF[1, i] = np.mean(RTF[1, (i - 1) * rN:(i + 1) * rN - 1])

In [236]:
p.plot(Kp[0], Kp[1])
p.plot(Ki[0], Ki[1])
p.plot(Kd[0], Kd[1])

In [350]:
import time
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')
p.close('all')

FRAD = np.nan_to_num(np.array([NECH[0], NRTF[1] / (NECH[1] / 1.e3)]))

T = T0
L = np.shape(FRAD)[1]
dT = FRAD[0, 1] - FRAD[0, 1]
END = L

P, I, D = .1, 1. / 60., .01

pid = PID(P, I, D, FRAD[0, 0])
pid.SetPoint = 0.9  # frad
pid.setSampleTime(dT)  # dT

feedback = FRAD[1, 0]
time_list = np.linspace(.0, T, L)

setpoint_list = np.zeros(L)
feedback_list = np.zeros(L)
adjust_list = np.zeros(L)
output_list = np.zeros(L)

prop_list = np.zeros(L)
integ_list = np.zeros(L)
diff_list = np.zeros(L)
error_list = np.zeros(L)

for i in range(END):
    t = time_list[i]
    setpoint_list[i] = pid.SetPoint

    if t < .5:
        prop_list[i] = np.nan
        integ_list[i] = np.nan
        diff_list[i] = np.nan
        feedback_list[i] = FRAD[1, i]
        continue

    pid.update(
        feedback, t,
        Kp_in=Kp[1, i] / np.max(Kp[1]),
        Ki_in=Ki[1, i] / np.max(Ki[1]))
    #     Kd_in=Kd[1, i] / np.max(Kd[1]))

    output_list[i] = pid.output
    error_list[i] = pid.last_error

    prop_list[i] = pid.PTerm
    integ_list[i] = pid.ITerm
    diff_list[i] = pid.DTerm

    if i == 0:
        feedback = FRAD[1, i]
    else:
        # feedback = FRAD[1, i] + (
        #     np.abs(.4 - feedback_list[i - 1]) * output_list[i])
        feedback = FRAD[1, i] + (
            np.abs(.4 - feedback_list[i - 1]) *
            np.exp(-1. * (t - time_list[i - 50])) * output_list[i])
    feedback_list[i] = feedback

p.plot(time_list, setpoint_list, c='grey', ls='-.')
p.plot(time_list, FRAD[1], c='r', ls='-.')
p.plot(time_list, feedback_list, c='b')
# p.plot(time_list, output_list, c='k', ls='-.')
# p.plot(time_list, prop_list, c='orange')  # .clip(0.0) / np.max(np.nan_to_num(prop_list)), c='orange')
# p.plot(time_list, integ_list, c='purple')  # .clip(0.0) / np.max(np.nan_to_num(integ_list)), c='purple')
# p.plot(time_list, diff_list, c='cyan')  # .clip(0.0) / np.max(np.nan_to_num(diff_list)), c='cyan')
# p.plot(np.abs(.4 - feedback_list), c='k')
# p.ylim(-.5, 5.)

In [ ]:
b_res = feedback_list
b_kp = prop_list
b_ki = integ_list
b_kd = diff_list

In [276]:
a_time = time_list
a_res = feedback_list
a_kp = prop_list
a_ki = integ_list
a_kd = diff_list

In [282]:
fig, ax = p.subplots(1, 1)  # , sharey=True)

colors = ['r', 'b', 'g', 'purple']
ax.set_ylabel('PID [a.u.]')

# ax[0].set_title('K$_{p}$:K$_{i}$:K$_{d}$ = 1:2/3:0')
# ax[1].set_title('K$_{p}$:K$_{i}$:K$_{d}$ = 1:2/5:3/100')
ax.set_title('K$_{p}$:K$_{i}$:K$_{d}$ = 1:1/6:10, N=30')

# for axis in ax:
axis = ax
axis.set_xlim(-.1, T0)
axis.set_xlabel('time [s]')
axis.plot(
    [-.2, T0 + .1], [0.9, 0.9],
    c='grey', ls='-.', alpha=.7)

ax.plot(time_list, a_kp.clip(.0) / np.max(np.nan_to_num(a_kp)), c='r', label='K$_{p}$')
ax.plot(time_list, a_ki.clip(.0) / np.max(np.nan_to_num(a_ki)), c='b', label='K$_{i}$')
ax.plot(time_list, a_kd / np.max(np.nan_to_num(a_kd)), c='g', label='K$_{d}$')
ax.plot(time_list, a_res, c='k', label='f$_{rad}$', ls='-.')
# ax.legend(bbox_to_anchor=(.45, .4), shadow=False, ncol=2)

# ax[1].plot(time_list, b_kp / np.max(np.nan_to_num(b_kp)), c='r', label='K$_{p}$')
# ax[1].plot(time_list, b_ki.clip(.0) / np.max(np.nan_to_num(b_ki)), c='b', label='K$_{i}$')
# ax[1].plot(time_list, b_kd.clip(.0) / np.max(np.nan_to_num(b_kd)), c='g', label='K$_{d}$')
# ax[1].plot(time_list, b_res, c='k', label='f$_{rad}$', ls='-.')
# ax[1].legend(bbox_to_anchor=(.475, .7), shadow=False, ncol=2)

# ax.plot(time_list, b_kp / np.max(np.nan_to_num(b_kp)), c='r', label='K$_{p}$')
# ax.plot(time_list, b_ki.clip(.0) / np.max(np.nan_to_num(b_ki)), c='b', label='K$_{i}$')
# ax.plot(time_list, b_kd.clip(.0) / np.max(np.nan_to_num(b_kd)), c='g', label='K$_{d}$')
# ax.plot(time_list, b_res, c='k', label='f$_{rad}$', ls='-.')
# ax.legend(bbox_to_anchor=(.5, .65),
#           shadow=False, ncol=2)

fig.set_size_inches(6., 3.5)
fig.savefig('pid_20181010032_fix.pdf', dpi=169.)
p.close('all')

In [47]:
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

import matplotlib as mpl
import matplotlib.pyplot as p
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
import numpy as np

from scipy.spatial import ConvexHull

import dat_lists
info = dat_lists.geom_dat_to_json()

import os
import mesh_2D

import invert_main as invert
import os
import mfr2D_accessoires as mfr2D

os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')
rG, zG = mesh_2D.store_read_mesh2D(
    name='EIM_beta000_sN8_30x20x150_1.3')['values']['fs']['108.']

import poincare
import poincare_plots

name = 'phi108.0_res10_conf_lin'
islands = poincare.store_read(
    name=name, arg='islands1_')

(raw_volume, old_volume, volume, line_sections2D,
 line_sections3D, emissivity2D, emissivity3D,
 factors, reff, pos_lofs, minor_radius, reff_LoS,
 camera_geometry, interpolated_geometry, LoS) = invert.main(
        nPhi=30,
        nL=150,
        nFS=20,
        vpF=1.3,
        N=[2, 4],  # 4
        tilt_deg=1.,
        error_scale=0.0001,  # 0.1 mm (0.5 mm)
        VMID='EIM_000',
        interp_method='square',  # triang
        new_type=None,  # 'HBCm'
        cartesian=False,
        add_camera=False,
        artificial_HBCm=False,
        fix_LoS=False,
        centered=False,
        symmetric=False,
        tilt=False,
        random_error=False,
        plot=False,
        debug=False)

rgrid, zgrid = mfr2D.reform_gridfile(
    program='20181010.032', kani=[1., 3.], nigs=1, grid_nt=150,
    grid_nr=20, debug=False, add_camera=False,
    RGS=False, phantom=False, nVals=[10, 10],
    reduced=True, no_ani=4, strgrid='sN8_30x20x150_1.3')[:2]

		\\ loaded mesh2D  0.00MB
		\\ load islands1  (4, 15, 100) 0.05MB
	Creating fluxsurface library for EIM_beta000
                FLUXSURFACE: beta-scan with bell-shaped pressure profile ~ (1-s)^2; peaking
                             factor = 3; runs with _l6ns151 have an enlarged volume - cover
                             island region EIM configuration (ref. configuration A: standard,
                             planned OP1.2 configuration). Flux surface geometry of EIM
                             configuration corresponds to the experimental EJM configuration
		Loading from: EIM_beta000_fs_data.json
		Splitting detectors and slit 2x4 times
		>> End result supposed to have 8 squares
	Constructing lines of sight...
		\\ load xy slope  (128, 64) 0.06MB
		\\ load xy constant  (128, 64) 0.06MB
		\\ load xy range  (128, 64, 2) 0.13MB
		\\ load xy line  (128, 64, 2) 0.13MB
		\\ load xz slope  (128, 64) 0.06MB
		\\ load xz constant  (128, 64) 0.06MB
		\\ load xz range  (128, 64, 2) 0.13M

In [63]:
fig, ax = p.subplots(1)

lines = LoS['values']['rz_plane']
T = np.shape(lines['line'])[1]

poincare_plots.plot_div(
    axis=ax, phi=108.,
    alpha=0.66, c='k', ls='-.',
    lw=0.75)
poincare_plots.PlotPoincare(
    axis=ax, surfaces=islands, alpha=0.5,
    col='k', style='-', marker='None',
    size=0.25, width=0.5)

ax.plot(  # LCFS
    rG[0], zG[0],
    c='k', ls='None',
    marker='x',
    markersize=4.)

channels = [61, 65, 73, 78, 86]
# channels = [14, 16, 7, 3, 0]

c_m = mpl.colors.LinearSegmentedColormap.from_list(
    'mycolors', ['red', 'blue'])
norm = mpl.colors.Normalize(vmin=0, vmax=len(channels))
s_m = cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])

length = [1., 2.1, 1.7, 1.8, 1.9]
# length = [4.7, 4.7, 4.5, 3.8, 2.9]

i = 0
for i, ch in enumerate(channels):
    pointsr, pointsz = [], []

    for k in range(T):
        lr, lz = lines['range'][ch, k], \
            lines['line'][ch, k]

        x = np.linspace(lr[0], lr[1], 1000)
        y = np.linspace(lz[0], lz[1], 1000)
        ind = np.where(
            ((x - x[0])**2 +
            (y - y[0])**2) < length[i])[0]
        lr, lz = [x[ind][0], x[ind][-1]], \
            [y[ind][0], y[ind][-1]]

        for r in lr:
            pointsr.append(r)
        for z in lz:
            pointsz.append(z)

    points = np.array([
        pointsr, pointsz]).transpose()
    hull = ConvexHull(points)
    ax.fill(
      points[hull.vertices,0],
      points[hull.vertices,1],
      color=s_m.to_rgba(i),
      alpha=0.5)

    ax.text(
        lr[-1] + 0.05, lz[-1] + 0.05, '#' + str(ch),
        color=s_m.to_rgba(i))

sch = 70
pointsr, pointsz = [], []

for k in range(T):
    lr, lz = lines['range'][sch, k], \
        lines['line'][sch, k]

    x = np.linspace(lr[0], lr[1], 1000)
    y = np.linspace(lz[0], lz[1], 1000)
    ind = np.where(
        ((x - x[0])**2 +
        (y - y[0])**2) < length[i])[0]
    lr, lz = [x[ind][0], x[ind][-1]], \
        [y[ind][0], y[ind][-1]]

    for r in lr:
        pointsr.append(r)
    for z in lz:
        pointsz.append(z)

points = np.array([
    pointsr, pointsz]).transpose()
hull = ConvexHull(points)
ax.fill(
    points[hull.vertices,0],
    points[hull.vertices,1],
    color='k', alpha=0.5)

ax.text(lr[-1] + 0.05, lz[-1] + 0.05,
        '#' + str(sch), color='k')

ax.text(4.4, .35, 'LCFS')
ax.axis('off')
fig.set_size_inches(5., 4.)
fig.tight_layout()
fig.savefig('20181010_32_channels.pdf', dpi=169.)
# fig.savefig('20180920_29_31_channels.pdf', dpi=169.)
p.close('all')

In [446]:
import importlib
import dat_lists
import webapi_access as api

import matplotlib.pyplot as p
import matplotlib as mpl
from matplotlib import cm

importlib.reload(dat_lists)
importlib.reload(api)

XPID = '20180920.032'  # '20180920.032'
program_info, req = api.xpid_info(program=XPID)
start = str(program_info['programs'][0]['from'])
stop = str(program_info['programs'][0]['upto'])

t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

names = ['CIII 19', 'CIII 53', 'CIII 57']
    # 'CIII 5', 'CIII 19', 'CIII 24', 'CIII 29',  # 0 - 3
    # 'CIII 36', 'CIII 41', 'CIII 53', 'CIII 57',  # 4 - 7
    # 'CII 35', 'CIV 20']  # 8 - 12

L = []
for name in names:
    foo = api.download_single(
        api.download_link(name=name),
        program_info=program_info,
        start_POSIX=start, stop_POSIX=stop)
    L.append(np.array([
        [(t - t0) / 1e9 for t in foo['dimensions']],
        foo['values']]))

In [447]:
L0920023 = L

In [445]:
L1010032 = L

In [457]:
N = 2000
# photons / (cm^2 sr), 465 nm
planck = 6.626e-34  # joule s
c = 299792458  # m / s
lambd = 465e-9
f = (planck * c / lambd) * 100**2


fig, ax = p.subplots(1, 2)  # , sharex=True)
for i, L in enumerate([L1010032, L0920023]):
    selection = L  # [L[1], L[6], L[7]]
    R = np.linspace(.0, 1., 3)
    R, T = np.meshgrid(R, L[0][0][::N])

    C = np.array([
        np.convolve(f * T[1], np.ones((N,)) / N, mode='same')[::N]
        for T in selection])

    CS = ax[i].contourf(T, R, C.transpose().clip(.0), 10, cmap=cm.viridis)
    cb = fig.colorbar(CS, ax=ax[i])

    ax[i].set_xlabel('time [s]')
    ax[i].set_ylabel('position [a.u.]')
    cb.set_label('brigh. [Wm$^{-3}$sr$^{-1}$]')
    labels = [item.get_text() for item in ax[i].get_yticklabels()]
    empty_string_labels = [''] * len(labels)
    ax[i].set_yticklabels(empty_string_labels)

ax[0].set_xlim(-.1, 9.5)    
ax[1].set_xlim(-.1, 10.5)
ax[0].text(.1, .9, 'plasma', color='white')
ax[0].text(6.5, .05, 'divertor', color='white')

fig.set_size_inches(9., 3.)
fig.tight_layout()
fig.savefig('20181010_032_0920_023_cIII_contour.pdf', dpi=169.)
p.close('all')

In [435]:
import importlib
import dat_lists
import webapi_access as api

import matplotlib.pyplot as p
import matplotlib as mpl
from matplotlib import cm

importlib.reload(dat_lists)
importlib.reload(api)

XPID = '20181010.032'
program_info, req = api.xpid_info(program=XPID)
start = str(program_info['programs'][0]['from'])
stop = str(program_info['programs'][0]['upto'])

t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

names = ['HAlpha 13', 'HAlpha 37', 'HAlpha 56']
L = []
for name in names:
    foo = api.download_single(
        api.download_link(name=name),
        program_info=program_info,
        start_POSIX=start, stop_POSIX=stop)
    L.append(np.array([
        [(t - t0) / 1e9 for t in foo['dimensions']],
        foo['values']]))

In [436]:
N = 2000
# photons / (cm^2 sr), 465 nm
planck = 6.626e-34  # joule s
c = 299792458  # m / s
lambd = 656.3e-9
f = (planck * c / lambd) * 100**2

selection = [L[0], L[1], L[2]]
R = np.linspace(.0, 1., 3)
R, T = np.meshgrid(R, L[0][0][::N])
C = np.array([
    np.convolve(f * T[1], np.ones((N,)) / N, mode='same')[::N]
    for T in selection])

fig, ax = p.subplots()
CS = ax.contourf(T, R, C.transpose().clip(.0), 10, cmap=cm.viridis)
cb = fig.colorbar(CS, ax=ax)

ax.set_xlim(-.1, 11.)
ax.set_xlabel('time [s]')
ax.set_ylabel('position [a.u.]')
cb.set_label('brigh. [Wm$^{-3}$sr$^{-1}$]')
ax.text(9., .9, 'plasma', color='white')
ax.text(8.5, .1, 'divertor', color='white')
labels = [item.get_text() for item in ax.get_yticklabels()]
empty_string_labels = [''] * len(labels)
ax.set_yticklabels(empty_string_labels)

fig.set_size_inches(6., 3.)
fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_hAlpha_contour.pdf')  # , dpi=169.)
p.close('all')